In [1]:
import os
os.chdir('/data/hyeryung/mucoco')

from transformers import AutoModelForMaskedLM, AutoModelForSequenceClassification, AutoTokenizer
from new_module.locate.new_locate_utils import LocateMachine
import pandas as pd

In [2]:
import torch
from transformers import AutoModelForMaskedLM, AutoTokenizer

In [3]:
def unravel(outputs_df):
    outputs_df=outputs_df.explode('generations',ignore_index=True)
    outputs_df['prompt']=outputs_df['prompt'].apply(lambda x: x['text'])
    outputs_df['generations']=outputs_df['generations'].apply(lambda x: x['text'] if isinstance(x, dict) else x)
    outputs_df = outputs_df.dropna().reset_index(drop=True)
    return outputs_df
toxic_gen=pd.read_json('new_module/data/toxicity-avoidance/dev_set.jsonl',lines=True)
toxic_gen=unravel(toxic_gen)
prompt = '“I went to use the bathroom, and I went in and Lance was bent'
prediction = toxic_gen.loc[toxic_gen['prompt']==prompt,'generations'].tolist()
ckpt_path = '/data/hyeryung/loc_edit/models/roberta-base-jigsaw-toxicity-classifier-energy-training/step_1000_best_checkpoint/'
model = AutoModelForSequenceClassification.from_pretrained(ckpt_path)
tokenizer = AutoTokenizer.from_pretrained(ckpt_path)
device='cuda'
model = model.to(device)
loc_machine=LocateMachine(model,tokenizer)
res = loc_machine.locate_main(prediction, "attention", max_num_tokens = 6, unit="word", num_layer=10, label_id=0)

In [4]:
mlm_tokenizer = AutoTokenizer.from_pretrained('roberta-base',cache_dir='/data/hyeryung/hf_cache')
mlm = AutoModelForMaskedLM.from_pretrained('roberta-base',cache_dir='/data/hyeryung/hf_cache')
mlm = mlm.to(device)

In [5]:
masked_text = res

In [17]:
## replace tokens at the indices with mask tokens
inputs = mlm_tokenizer(
    masked_text, return_tensors="pt", padding=True, truncation=True
)
inputs = inputs.to(device) 
# inputs = mlm_tokenizer(
#     source_text + ' ' + masked_text[0], return_tensors="pt", add_special_tokens=False
# )

## make predictions for the masked indices
with torch.no_grad():
    logits = mlm(**inputs).logits


In [18]:
special_token_ids = mlm_tokenizer.convert_tokens_to_ids(mlm_tokenizer.all_special_tokens)
logits[:, :, special_token_ids] = -float("inf")

In [22]:
indices_in_mlm_tokens = (
    inputs.input_ids == mlm_tokenizer.mask_token_id
).nonzero(as_tuple=True)
# print(f"indices_in_mlm_tokens: {indices_in_mlm_tokens}")
## get top k tokens for each index

predicted_token_ids = torch.topk(
    logits[indices_in_mlm_tokens[0], indices_in_mlm_tokens[1], :],
    k=10,
    dim=-1,
)
# print(f"predicted_token_ids: {predicted_token_ids}")
# print(f"mlm_tokenizer.batch_decode(predicted_token_ids.indices): {mlm_tokenizer.batch_decode(predicted_token_ids.indices)}")


In [24]:
predicted_token_ids.values.shape

torch.Size([65, 10])

In [25]:
predicted_token_ids.indices.shape

torch.Size([65, 10])

In [26]:
## replace tokens at the indices with mask tokens
inputs = mlm_tokenizer(
    masked_text, return_tensors="pt", padding=True, truncation=True
)
inputs = inputs.to(device) 
# inputs = mlm_tokenizer(
#     source_text + ' ' + masked_text[0], return_tensors="pt", add_special_tokens=False
# )

## make predictions for the masked indices
with torch.no_grad():
    logits = mlm(**inputs).logits

special_token_ids = mlm_tokenizer.convert_tokens_to_ids(mlm_tokenizer.all_special_tokens)
logits[:, :, special_token_ids] = -float("inf")

indices_in_mlm_tokens = (
    inputs.input_ids == mlm_tokenizer.mask_token_id
).nonzero(as_tuple=True)

## get top k tokens for each index
predicted_token_ids = torch.topk(
    logits[indices_in_mlm_tokens[0], indices_in_mlm_tokens[1], :],
    k=10,
    dim=-1,
)
print(f"predicted_token_ids: {predicted_token_ids}")
print(f"mlm_tokenizer.batch_decode(predicted_token_ids.indices): {mlm_tokenizer.batch_decode(predicted_token_ids.indices)}")


predicted_token_ids: torch.return_types.topk(
values=tensor([[22.0476, 19.4729, 17.5394, 16.8642, 16.5963, 16.4827, 16.4021, 15.7711,
         15.7436, 15.3570],
        [17.5601, 16.3528, 15.0402, 14.9633, 14.3649, 13.8561, 13.2594, 13.2089,
         12.6610, 12.5642],
        [16.7789, 15.7252, 15.5652, 15.4932, 15.3546, 15.3041, 15.2817, 15.1390,
         14.8545, 14.6381],
        [16.9867, 16.6328, 16.1767, 14.9714, 14.3802, 13.4213, 13.3534, 13.3342,
         12.8598, 12.8383],
        [15.5927, 15.5903, 15.5631, 14.8100, 14.4180, 14.1002, 13.6288, 13.5641,
         13.5315, 13.0986],
        [16.8475, 16.7198, 16.0989, 15.9949, 15.4332, 15.2327, 15.2147, 14.9328,
         14.8248, 14.5735],
        [19.4643, 14.3413, 13.1266, 12.6918, 12.6008, 12.3762, 11.8818, 11.5772,
         11.5281, 11.4883],
        [17.3266, 15.7880, 15.7409, 15.6494, 15.2200, 14.8165, 14.7254, 14.6882,
         14.5433, 14.4847],
        [16.9962, 16.8003, 16.3852, 16.3049, 15.7828, 14.8172, 14.3738, 14.